In [1]:
from ftplib import FTP
from io import BytesIO
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Function to download spectrum from FTP
def download_spectrum_ftp(url):
    from urllib.parse import urlparse

    parsed_url = urlparse(url)
    ftp_host = parsed_url.hostname
    ftp_file_path = parsed_url.path

    ftp = FTP(ftp_host)
    ftp.login()

    file_data = BytesIO()
    ftp.retrbinary(f"RETR {ftp_file_path}", file_data.write)
    ftp.quit()

    file_data.seek(0)
    return file_data.read()

# Intrinsic luminosity dictionary
intrinsic_lum = {
    'O5': 846000, 'O9': 95000,
    'B0': 20000, 'B1': 4600,
    'A0': 22, 'A2': 18,
    'F0': 4.3, 'F2': 3.3,
    'G0': 1.3, 'G2': 1,
    'K0': 0.54, 'K2': 0.38,
    'M0': 0.069, 'M1': 0.064
}

# Spectra URLs dictionary with explicit spectral type mapping
spectra_urls = {
    "O5": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/uko5v.fits",
    "O9": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/uko9v.fits",
    "B0": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukb0v.fits",
    "B1": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukb1v.fits",
    "A0": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/uka0v.fits",
    "A2": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/uka2v.fits",
    "F0": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukf0v.fits",
    "F2": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukf2v.fits",
    "G0": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukg0v.fits",
    "G2": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukg2v.fits",
    "K0": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukk0v.fits",
    "K2": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukk2v.fits",
    "M0": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukm0v.fits",
    "M1": "ftp://ftp.eso.org/web/sci/observing/tools/standards/IR_spectral_library/ukm1v.fits"
}

# Download the spectra data
spectra = {key: download_spectrum_ftp(url) for key, url in spectra_urls.items()}

In [ ]:
# Masas para categorizar Estrellas
def Mclassifier(mass):
    if mass > 16:
        return "O"
    elif mass > 2.1:
        return "B"
    elif mass > 1.4:
        return "A"
    elif mass > 1.04:
        return "F"
    elif mass > 0.8:
        return "G"
    elif mass > 0.45:
        return "K"
    else:
        return "M"
    
# DISCRTEIZAR A 14 GYR, cambiar esto, como elegimos esta edad
universe_age = 14


def lum(mass):
    if mass > 55:
        return 3200 * mass
    elif mass > 2:
        return mass ** 4
    elif mass > 0.43:
        return 1.4 * mass ** 3.5
    else:
        return 0.23 * mass ** 2.3
 

def STARage(mass):
    #returns: time in SP, estimated age

    def SP_time(mass):
        sp_Sun = 10.0
        return sp_Sun / (mass ** 3)
    
    if mass > 5:
        return SP_time(mass), SP_time(mass)
    else: 
        return SP_time(mass) , SP_time(mass) + 1.1


spectral_types = ['O', 'B', 'A', 'F', 'G', 'K', 'M']

def read_spectrum_from_fits(data):
    with fits.open(BytesIO(data)) as hdul:
        spectrum_data = hdul[0].data
        header = hdul[0].header
        wavelengths = np.arange(header['CRVAL1'], header['CRVAL1'] + header['CDELT1'] * len(spectrum_data), header['CDELT1'])
        return wavelengths[:len(spectrum_data)], spectrum_data

# OCUPA LOS ESPECTROS REALES
def get_star_spectrum(spectral_type):
    if spectral_type in spectra:
        data = spectra[spectral_type][0]
        wavelengths, spectrum = read_spectrum_from_fits(data)
        return wavelengths, spectrum
    else:
        raise ValueError(f"Spectral type {spectral_type} not available in spectra.")
    

# Function to calculate the total luminosity of the spectrum
def tot_lum(wavelength, spectrum):
    luminosity = np.trapz(spectrum, x=wavelength)
    return luminosity


def normflux(wavelenght, spectrum):
    total_lum = tot_lum(wavelenght, spectrum)
    return spectrum / total_lum



    

In [ ]:
# el weight me lo da la imf
class STAR:
    def __init__(self, M, W, t):
        self.mass = M
        self.Tbirth = t
        self.weight = W # weight in relation of how many stars of this type are made
        self.lum = lum(M)
        self.Mtype = Mclassifier(M)
        self.SPtime = STARage(M)[0]
        self.wavelenght = get_star_spectrum(self.Mtype)[0]
        self.spectrum = get_star_spectrum(self.Mtype)[1]
        self.flux = normflux(self.wavelenght, self.spectrum)

In [ ]:
#como agregar la IMF -> tiene una funcion generate masses que me sirve
class GALAXY:
    def __init__(self, M, min, max, IMF):
        self.stars = []
        self.minMass = min
        self.maxMAss = max
        self.tot_mass = M
        self.IMF = IMF
    
    def Star_Formation(t):
        #####
        mlist = IMF()
        for i in range(mlist):
            self.stars.append(STAR(mlist[i],IMF.pdf[i],t))

        
        


